In [7]:
from tests.CONFIG import api_key
from requests import request, get, post

url_by_id = "https://api.kinopoisk.dev/v1/movie/possible-values-by-field"
response = get(url=url_by_id, headers={"X-API-KEY": api_key}, params={"field": "genres.name"})

In [8]:
print(response.json())
print(type(response.json()))

[{'name': 'аниме', 'slug': 'anime'}, {'name': 'биография', 'slug': 'biografiya'}, {'name': 'боевик', 'slug': 'boevik'}, {'name': 'вестерн', 'slug': 'vestern'}, {'name': 'военный', 'slug': 'voennyy'}, {'name': 'детектив', 'slug': 'detektiv'}, {'name': 'детский', 'slug': 'detskiy'}, {'name': 'для взрослых', 'slug': 'dlya-vzroslyh'}, {'name': 'документальный', 'slug': 'dokumentalnyy'}, {'name': 'драма', 'slug': 'drama'}, {'name': 'игра', 'slug': 'igra'}, {'name': 'история', 'slug': 'istoriya'}, {'name': 'комедия', 'slug': 'komediya'}, {'name': 'концерт', 'slug': 'koncert'}, {'name': 'короткометражка', 'slug': 'korotkometrazhka'}, {'name': 'криминал', 'slug': 'kriminal'}, {'name': 'мелодрама', 'slug': 'melodrama'}, {'name': 'музыка', 'slug': 'muzyka'}, {'name': 'мультфильм', 'slug': 'multfilm'}, {'name': 'мюзикл', 'slug': 'myuzikl'}, {'name': 'новости', 'slug': 'novosti'}, {'name': 'приключения', 'slug': 'priklyucheniya'}, {'name': 'реальное ТВ', 'slug': 'realnoe-TV'}, {'name': 'семейный',

In [9]:
countries = list(map(lambda x: x["name"], response.json()))

['аниме', 'биография', 'боевик', 'вестерн', 'военный', 'детектив', 'детский', 'для взрослых', 'документальный', 'драма', 'игра', 'история', 'комедия', 'концерт', 'короткометражка', 'криминал', 'мелодрама', 'музыка', 'мультфильм', 'мюзикл', 'новости', 'приключения', 'реальное ТВ', 'семейный', 'спорт', 'ток-шоу', 'триллер', 'ужасы', 'фантастика', 'фильм-нуар', 'фэнтези', 'церемония']


from movies_app.models import Genre

for c in countries:
    obj = Country(name=c)
    obj.save()

In [8]:
from requests import request, get, post
from tests.CONFIG import api_key

url = "https://api.kinopoisk.dev/v1.4/movie"
response = get(url, headers={"X-API-KEY": api_key}, params={"lists": "top250", "limit": 250})

In [9]:
from pprint import pprint

data = response.json()['docs']

In [10]:
db_data = []
for line in data:
    pprint(line)
    d = {
        "name": line["names"][0]['name'],
        "year": line['year'],
        "description": line['description'],
        "slogan": line.get("slogan", None),
        "duration": line['movieLength'],
        "rate": line['ageRating'],
        "kinopoisk_id": line['id'],
        "rating_kp": line['rating']['kp'],
        "rating_imdb": line['rating']['imdb'],
        "poster": line['poster']['url']
    }
    db_data.append(d)
    
pprint(db_data)

{'ageRating': 18,
 'alternativeName': 'Intouchables',
 'backdrop': {'previewUrl': 'https://image.openmoviedb.com/tmdb-images/w500/bGksau9GGu0uJ8DJQ8DYc9JW5LM.jpg',
              'url': 'https://image.openmoviedb.com/tmdb-images/original/bGksau9GGu0uJ8DJQ8DYc9JW5LM.jpg'},
 'countries': [{'name': 'Франция'}],
 'description': 'Пострадав в результате несчастного случая, богатый аристократ '
                'Филипп нанимает в помощники человека, который менее всего '
                'подходит для этой работы, – молодого жителя предместья '
                'Дрисса, только что освободившегося из тюрьмы. Несмотря на то, '
                'что Филипп прикован к инвалидному креслу, Дриссу удается '
                'привнести в размеренную жизнь аристократа дух приключений.',
 'enName': None,
 'genres': [{'name': 'драма'}, {'name': 'комедия'}, {'name': 'биография'}],
 'id': 535341,
 'isSeries': False,
 'logo': {'url': 'https://avatars.mds.yandex.net/get-ott/1531675/2a0000017f0262661cde61dc260cb86

In [20]:
from requests import get
from CONFIG import api_key
from movies_app.models import *


url_for_film = "https://api.kinopoisk.dev/v1.4/movie"
for film in db_data:
    film_id = film['kinopoisk_id']
    response = get(url, headers={"X-API-KEY": api_key},params={"id": film_id, "limit": 1, "selectFields": ["countries", "persons", "id", "genres", "budget"]})
    
    data = response.json()
    data = data['docs'][0]
    
    genres = data['genres']
    genres = list(map(lambda x: x['name'], genres))
    
    budget = data['budget']
    if not budget:
        budget = None
    else:
        budget = f"{budget['value']}{budget['currency']}"
    
    new_movie = Movie(
        name = film["name"],
        year = film['year'],
        description = film['description'],
        duration = film['duration'],
        budget = budget,
        rate = film['rate'],
        kinopoisk_id = film['kinopoisk_id'],
        rating_kp = film['rating_kp'],
        rating_imdb = film['rating_imdb'],
        poster = film['poster']
    )
    movie = new_movie.save()
    movie_id = movie.id
    
    for g in genres:
        genre_id = Genre.objects.filter(name=g).id      
        new_movie_genre = MovieGenre(
            movie_id = movie_id,
            genre_id = genre_id
        )
        new_movie_genre.save()
    country = data['countries']
    country = list(map(lambda x: x['name'], country))
    for c in country:
        country_id = Country.objects.filter(name=c).id
        new_country_movie = MovieCountry(
            movie_id = movie.id,
            country_id = country_id
        ) 
        new_country_movie.save()
    
    persons = data['persons']    

    for p in persons:
        prof = p['profession']
        if prof in ["режиссеры", "актеры"]:
            name = p['name']
            photo = p['photo']
            person_id = p['id']
        else:
            continue
        url_person = "https://api.kinopoisk.dev/v1.4/person"
        p_data = get(url=url_person, headers={"X-API-KEY": api_key}, params={"id": person_id}).json()
        country = p_data['birthPlace'][1]['value']
        sex = p_data['sex']
        year = p_data['birthday'].split('-')[0]
        country_id = Country.objects.filter(name=country).id
        
        new_person = Person(
            name = name,
            year = int(year),
            country_id = country_id,
            sex = sex,
            photo = photo
        )
        pers = new_person.save()
        
        person_id = pers.id
        prof_id = Profession.objects.filter(name=prof).id
        new_person_profession = PersonProfession(
            person_id = person_id,
            profession_id = profession_id
        )
        new_person_profession.save()
        
        new_person_movie = MoviePerson(
            movie = movie_id,
            person = person_id
        )

    
print(data)

{'docs': [{'id': 535341, 'budget': {'value': 9500000, 'currency': '€'}, 'genres': [{'name': 'драма'}, {'name': 'комедия'}, {'name': 'биография'}], 'countries': [{'name': 'Франция'}], 'persons': [{'id': 71427, 'photo': 'https://st.kp.yandex.net/images/actor_iphone/iphone360_71427.jpg', 'name': 'Франсуа Клюзе', 'enName': 'François Cluzet', 'description': 'Philippe', 'profession': 'актеры', 'enProfession': 'actor'}, {'id': 41644, 'photo': 'https://st.kp.yandex.net/images/actor_iphone/iphone360_41644.jpg', 'name': 'Омар Си', 'enName': 'Omar Sy', 'description': 'Driss', 'profession': 'актеры', 'enProfession': 'actor'}, {'id': 57174, 'photo': 'https://st.kp.yandex.net/images/actor_iphone/iphone360_57174.jpg', 'name': 'Анн Ле Ни', 'enName': 'Anne Le Ny', 'description': 'Yvonne', 'profession': 'актеры', 'enProfession': 'actor'}, {'id': 868557, 'photo': 'https://st.kp.yandex.net/images/actor_iphone/iphone360_868557.jpg', 'name': 'Одри Флёро', 'enName': 'Audrey Fleurot', 'description': 'Magalie'

In [21]:

print(genres)

['драма', 'комедия', 'биография']


In [25]:

print(budget)

9500000€


In [28]:

print(country)

['Франция']


In [29]:
pprint(persons)

[{'description': 'Philippe',
  'enName': 'François Cluzet',
  'enProfession': 'actor',
  'id': 71427,
  'name': 'Франсуа Клюзе',
  'photo': 'https://st.kp.yandex.net/images/actor_iphone/iphone360_71427.jpg',
  'profession': 'актеры'},
 {'description': 'Driss',
  'enName': 'Omar Sy',
  'enProfession': 'actor',
  'id': 41644,
  'name': 'Омар Си',
  'photo': 'https://st.kp.yandex.net/images/actor_iphone/iphone360_41644.jpg',
  'profession': 'актеры'},
 {'description': 'Yvonne',
  'enName': 'Anne Le Ny',
  'enProfession': 'actor',
  'id': 57174,
  'name': 'Анн Ле Ни',
  'photo': 'https://st.kp.yandex.net/images/actor_iphone/iphone360_57174.jpg',
  'profession': 'актеры'},
 {'description': 'Magalie',
  'enName': 'Audrey Fleurot',
  'enProfession': 'actor',
  'id': 868557,
  'name': 'Одри Флёро',
  'photo': 'https://st.kp.yandex.net/images/actor_iphone/iphone360_868557.jpg',
  'profession': 'актеры'},
 {'description': 'La DRH société de courses',
  'enName': 'Joséphine de Meaux',
  'enProfes

In [3]:
data_persons = []
data_personsProf = {}
data_moviePersons = {}
data_movieCountry = {}
data_movieGenre = {}

NameError: name 'persons' is not defined

In [ ]:
data_persons = {
    "name": name
}